In [ ]:
from LiveAMP import *
self = AMP(**kwargs)
S = self.run()

In [3]:
S

overall_score  \
crse_code levl_code styp_code pred_code train_code trf_hash imp_hash clf_hash sim                  
_allcrse  ug        n         202108    202108     fa15     36d5     c244     0        39.527859   
                                                                              1        36.986022   
                                                                              2        35.394069   
                                                                              3        34.792033   
                                                                              4        40.327737   
...                                                                                          ...   
                              202308    202308     fa15     36d5     1d1f     25       19.996737   
                                                                              26       19.561244   
                                                                              27       19.833078   
                                                                              28       20.420894   
                                                                              29       19.934021   

                                                                                   train_score  \
crse_code levl_code styp_code pred_code train_code trf_hash imp_hash clf_hash sim                
_allcrse  ug        n         202108    202108     fa15     36d5     c244     0       20.11523   
                                                                              1       20.09026   
                                                                              2      19.989519   
                                                                              3      20.342492   
                                                                              4      20.760612   
...                                                                                        ...   
                              202308    202308     fa15     36d5     1d1f     25     10.298017   
                                                                              26     10.402366   
                                                                              27     10.238372   
                                                                              28     10.317374   
                                                                              29     10.343281   

                                                                                   test_score  \
crse_code levl_code styp_code pred_code train_code trf_hash imp_hash clf_hash sim               
_allcrse  ug        n         202108    202108     fa15     36d5     c244     0     19.412628   
                                                                              1     16.895762   
                                                                              2     15.404549   
                                                                              3     14.449541   
                                                                              4     19.567125   
...                                                                                       ...   
                              202308    202308     fa15     36d5     1d1f     25     9.698721   
                                                                              26     9.158879   
                                                                              27     9.594706   
                                                                              28     10.10352   
                                                                              29      9.59074   

                                                                                   actual  \
crse_code levl_code styp_code pred_code train_code trf_hash imp_hash clf_hash sim           
_allcrse  ug        n         202108    202108     fa15    

In [ ]:
S

In [ ]:
self.get('mlt')

In [ ]:
from LiveAMP import *
proj_code = 202408
path = {'styp_code': 'n', 'crse_code': '_allcrse'}
mlt = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/150/False/mlt.pkl')
M = mlt.query(f"styp_code==@path['styp_code'] & crse_code==@path['crse_code']").rsindex('pred_code').squeeze().rename(f'{proj_code}_projection')
A = pathlib.Path('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/150/False/Y_pred/'+join(path.values(),'/'))
for B in sorted(A.iterdir()):
    path['train_code'] = B.stem
    for C in sorted(B.iterdir()):
        path['trf_hash'] = C.stem
        for D in sorted(C.iterdir()):
            path['imp_hash'] = D.stem
            for E in sorted(D.iterdir()):
                path['clf_hash'] = E.stem
                dct = read(E)
                for path['sim'], (Y, ts) in enumerate(zip(dct['Y_pred'], dct['train_score'])):
                    print(path)
                    # A = Y.groupby('pred_code')
                    A = Y.addlevel('imp_hash', path['imp_hash']).addlevel('sim', path['sim'])
                    S = A.groupby([*path.keys(),'pred_code']).apply(lambda y: pd.Series({
                        'train_score': ts,
                        'test_score': (1 - f1_score(y['actual'], y['predicted'])) * 100,
                        'actual': y['actual'].sum(),
                        'predicted': y['predicted'].sum(),
                        })).prep()
                    S.insert(0, 'overall_score', S['train_score'] + S['test_score'])
                    S['error'] = S['predicted'] - S['actual']
                    S['error_pct'] = S['error'] / S['actual'] * 100
                    proj_mask = S.eval("pred_code==@proj_code")
                    S.loc[proj_mask, [c for c in S.columns if 'score' in c or 'error' in c]] = pd.NA
                    S = S.join(M*S.loc[proj_mask, 'predicted'].squeeze())
                    S.disp(10)
                    assert 1==2
                    S



In [ ]:
(5127-2731)/2731

In [ ]:
dct = dict()
lst = [1,2]
for dct['a'] in lst:
    print('hi')
dct


In [ ]:
A = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/150/False/Z_proc/n/all/all/fa15/9f20/all.pkl')
targ = '_allcrse'
clf_par = {
    'ensemble':True,
    'estimator_list':['lgbm','xgboost','rf','extra_tree','histgb'],
    'metric':'f1',
    'seed':42,
    'split_type':'stratified',
    'task':'classification',
    'time_budget': 60*10,
    'early_stop': True,
    'eval_method': 'cv',
    'n_splits': 5,
    'verbose':0}
cols = uniquify([*A['Z_proc'][0].filter(like='__').columns, '_allcrse_cur', targ+'_cur', targ])
Z = A['Z_proc'][0].filter(cols).copy()
trn_mask = Z.eval(f"pred_code==202308")
clf = fl.AutoML(**clf_par)
clf.fit(Z[trn_mask].drop(columns=targ), Z[trn_mask][targ], **clf_par, log_file_name=self.root_path / 'log.txt')
# plt.barh(clf.model.estimator.feature_name_, clf.model.estimator.feature_importances_)
# plt.show()
time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history = fl.automl.data.get_output_from_log(filename=self.root_path / "log.txt", time_budget=120)
plt.title("Learning Curve")
plt.xlabel("Wall Clock Time (s)")
plt.ylabel("Validation Accuracy")
plt.step(time_history, 1 - np.array(best_valid_loss_history), where="post")
plt.show()

In [ ]:
time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history = fl.automl.data.get_output_from_log(filename=self.root_path / "log.txt", time_budget=np.inf)
plt.title("Learning Curve")
plt.xlabel("Wall Clock Time (s)")
plt.ylabel("Validation Accuracy")
plt.step(time_history, 1 - np.array(best_valid_loss_history), where="post")
plt.show()

In [ ]:
from LiveAMP import *
A = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/150/False/Z_proc/n/all/all/fa15/9f20/all.pkl')
targ = '_allcrse'
cols = uniquify([*A['Z_proc'][0].filter(like='__').columns, '_allcrse_cur', targ+'_cur', targ])
Z = A['Z_proc'][0].filter(cols).copy()
trn_mask = Z.eval(f"pred_code==202308")

dt = 30
clf_par = {
    'ensemble':True,
    'estimator_list':[
        'lgbm',
        'xgboost',
        # 'rf',
        # 'extra_tree',
        # 'histgb',
        ],
    'metric':'f1',
    'seed':42,
    'split_type':'stratified',
    'task':'classification',
    'time_budget': 120,
    # 'time_budget': dt,
    'early_stop': True,
    'eval_method': 'cv',
    'n_splits': 5,
    'verbose': 0,
    'log_file_name': '/home/scook/institutional_data_analytics/admitted_matriculation_projection/admitted_matriculation_predictor/log.txt',
    'log_type': 'all',
    'X_train': Z[trn_mask].drop(columns=targ),
    'y_train': Z[trn_mask][targ],
    }

def plot():
    time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history = fl.automl.data.get_output_from_log(filename=clf_par['log_file_name'], time_budget=np.inf)
    plt.title("Learning Curve")
    plt.xlabel("Wall Clock Time (s)")
    plt.ylabel("Validation Accuracy")
    plt.step(time_history, 1 - np.array(best_valid_loss_history), where="post")
    plt.show()

clf = fl.AutoML(**clf_par)
clf.fit(**clf_par)
plot()
# for k in range(100):
#     clf_par['time_budget'] += dt
#     clf.retrain_from_log(**{k: clf_par[k] for k in {'log_file_name','X_train','y_train'}})
#     clf.fit(**clf_par)
#     plot()

In [ ]:
Y = (
    Z[targ].rename('actual').to_frame()
    .assign(predicted=clf.predict(Z.drop(columns=targ)))
    .assign(proba=clf.predict_proba(Z.drop(columns=targ))[:,1])
)
Y

In [ ]:
clf.predict(Z)

In [ ]:
clf.predict(Z.drop(columns=targ))

In [ ]:
Y = (
    Z[targ].rename('actual').to_frame()
    .assign(predicted=clf.predict(Z.drop(columns=targ)))
    .assign(proba=clf.predict_proba(Z.drop(columns=targ))[:,1])
)


In [ ]:
q = fl.AutoML()
q.fit?

In [ ]:
clf.retrain_from_log?

In [ ]:
clf.retrain_from_log

In [ ]:
vars(clf)

In [ ]:
plt.barh(clf.model.estimator.feature_name_, clf.model.estimator.feature_importances_)

In [ ]:
from flaml.automl.data import get_output_from_log

import matplotlib.pyplot as plt
import numpy as np

time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history = fl.automl.data.get_output_from_log(filename=self.root_path / "log.txt", time_budget=120)
plt.title("Learning Curve")
plt.xlabel("Wall Clock Time (s)")
plt.ylabel("Validation Accuracy")
plt.step(time_history, 1 - np.array(best_valid_loss_history), where="post")
plt.show()

In [ ]:
path = {'nm': 'Z_proc', 'styp_code': 'n', 'crse_code': '_allcrse', 'train_code': 202308, 'trf_hash': '8dec', 'imp_hash': '2438', 'clf_hash': '0213'}
dct = deepcopy(self.get(path | {'nm':'Z_proc', 'crse_code':'all', 'train_code':'all', 'clf_hash':'all'})) | {'Y_pred':[], 'train_score': []}
targ = path['crse_code']
# df = dct['Z_proc']
clf_par = self.clf_dict[path['clf_hash']]
clf_par['seed'] = self.random_state

clf_par['time_budget'] = 2
clf_par['estimator_list'] = ['lgbm', 'xgboost', 'xgb_limitdepth', 'rf', 'extra_tree']

# for imp, Z in enumerate(df):
# Z = df[0]
for Z in deepcopy(dct['Z_proc']):
    cols = uniquify([*Z.filter(like='__').columns, '_allcrse_cur', targ+'_cur', targ])
    Z = Z.filter(cols).copy().addlevel('crse_code', targ).addlevel('train_code', path['train_code']).addlevel('clf_hash', path['clf_hash']).prep().prep_category()
    trn_mask = Z.eval(f"pred_code==@path['train_code']")
    clf = fl.AutoML(**clf_par)
    # with warnings.catch_warnings(action='ignore'):
    clf.fit(X_train=Z[trn_mask].drop(columns=targ), y_train=Z[trn_mask][targ], **clf_par)
    print('done')
        #  eval_method='cv', n_splits=self.n_splits, **clf_par)

In [ ]:
{'nm': 'Z_proc', 'styp_code': 'n', 'crse_code': '_allcrse', 'train_code': 202308, 'trf_hash': '8dec', 'imp_hash': '2438', 'clf_hash': '0213'}: {'nm': 'Y_pred', 'styp_code': 'n', 'crse_code': '_allcrse', 'train_code': 202308, 'trf_hash': '8dec', 'imp_hash': '2438', 'clf_hash': '0213'}
path = dct['path'] |{'clf_hash': '5023', 'crse_code':'_allcrse', 'train_code':202308}
targ = path['crse_code']
clf_par = self.clf_dict[path['clf_hash']]
clf_par['seed'] = self.random_state
clf_par['time_budget'] = 2
clf_par['ensemble'] = {"final_estimator": LogisticRegression()}
print(clf_par)
Z = dct['Z_proc'][0]
cols = uniquify([*Z.filter(like='__').columns, '_allcrse_cur', targ+'_cur', targ])
Z = Z.filter(cols).copy().addlevel('crse_code', targ).addlevel('train_code', path['train_code']).addlevel('clf_hash', path['clf_hash']).prep().prep_category()
trn_mask = Z.eval(f"pred_code==@path['train_code']")
clf = fl.AutoML(**clf_par)
with warnings.catch_warnings(action='ignore'):
    clf.fit(Z[trn_mask].drop(columns=targ), Z[trn_mask][targ], eval_method='cv', n_splits=self.n_splits, **clf_par)

# Z

In [ ]:
fl.AutoML?

In [ ]:
X.groupby(['pred_code'])['__fafsa_app'].mean()

In [ ]:
self.trf_dict

In [ ]:
dct['holdout_score']

In [ ]:
hasher = lambda t: hashlib.shake_128(str(t).encode()).hexdigest(2)
for nm in ['trf','imp','clf']:
    print(nm)
    self[nm+'_list'] = cartesian({k: uniquify(v, key=str) for k,v in self[nm+'_grid'].items()})
    print(len(self[nm+'_list']))
    if nm == 'trf':
        self[nm+'_list'] = [ColumnTransformer([(c,t,["__"+c]) for c,t in trf.items() if t not in ['drop',None,'']], remainder='drop', verbose_feature_names_out=False) for trf in self[nm+'_list']]
        self[nm+'_dict'] = {hasher(t.transformers): t for t in self[nm+'_list']}
    else:
        self[nm+'_dict'] = {hasher(t): t for t in self[nm+'_list']}
    print(len(self[nm+'_dict']))


In [ ]:

hashlib.shake_128(str(x.transformers).encode()).hexdigest(2)

In [ ]:
for x in self['trf_list']:
    print(str(x).replace('\n','').replace(' ',''))

In [ ]:
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/53c7/c519/5742.pkl'
# fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/5afa/c519/5742.pkl'
# fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/b051/c519/5742.pkl'
# fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/b314/c519/5742.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/e69c/c519/5742.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/e858/c519/5742.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/ed94/c519/5742.pkl'
A = read(fn)
A.keys()
len(A['train_score'])

In [ ]:
1+1

In [ ]:
for k,v in self.trf_dict.items():
    print(k)
    print(str(v).replace('\n','').replace(' ',''))

In [ ]:
A = self.optimal['n']['_allcrse'][202308]
a = min(A, key=lambda x: x['score']['agg'])
a.keys()

# [0].keys()

In [ ]:
for path in cartesian({'nm': '', 'styp_code': self.styp_codes, 'crse_code': self.crse_codes, 'train_code': self.train_codes}, sort=False):
    s = self.get(path | {'nm': 'summary'})
s['53c7']['3d79']['5742'].keys()

In [ ]:
cartesian({'styp_code': self.styp_codes, 'crse_code': self.crse_codes, 'train_code': self.train_codes, 'trf_hash': self.trf_dict.keys(), 'imp_hash': self.imp_dict.keys(), 'clf_hash': self.clf_dict.keys()})


In [ ]:
self.terms[202108]['cur'].get('raw')

In [ ]:
self.get('reg_df')

In [ ]:
self.terms

In [ ]:
cartesian({'pred_code': self.pred_codes})

In [ ]:
import codetiming
x = 'hi'
with codetiming.Timer(text=x+"Elapsed time: {:0.4f} seconds"):
    for i in range(100000):
        pass

In [ ]:
from LiveAMP import *

# FLAGS().run()
self = AMP(**kwargs)
self.get('raw_df')
self.get('reg_df')
self.get('X')
self.get('Y')
self.get('mlt')
for styp_code in self.styp_codes:
    for crse_code in self.crse_codes:
        for train_code in self.train_codes:
            for trf_hash in self.trf_dict.keys():
                for imp_hash in self.imp_dict.keys():
                    for clf_hash in self.clf_dict.keys():
                        path = {'nm':'summary', 'styp_code': styp_code, 'crse_code':crse_code, 'train_code':train_code, 'trf_hash':trf_hash, 'imp_hash':imp_hash, 'clf_hash':clf_hash}
                        print(path)
                        dct = self.get(path)

In [ ]:
dct['rslt'].describe().T

In [ ]:
clf['Y_pred'][0]
clf.keys()
clf['2024_projection_202108']
clf.keys()
pd.DataFrame({k:v for k,v in clf.items() if 'score' in k or 'projection' in k})
# for k in filter(lambda k,v: 'score' in k, clf.items()):
#     print(k)

In [ ]:
def f(dct):
    dct.pop('a')

class A():
    def __init__(self):
        dct = {'a':3}
        f(dct)
        print(dct)
A()

In [ ]:
M = self.get('mlt').query(f"styp_code=='n' & crse_code=='_allcrse'").rsindex('pred_code').squeeze()
M
{k:v for k,v in M.items()}


In [ ]:
clf['rslt'].describe().T.disp(10)

In [ ]:
self.get('X').dtypes.disp(100)

In [ ]:
M = self.get('mlt').query(f"styp_code==@path['styp_code'] & crse_code==@path['crse_code']")
A = (clf['Y_pred'].query('imp==0')['predicted'].sum() * M).reset_index()[['pred_code','mlt']]
A.to_dict('records')
# ('index')

In [ ]:
M = self.get('mlt').query(f"styp_code==@path['styp_code'] & crse_code==@path['crse_code']")
M
# A = (clf['Y_pred'].query('imp==0')['predicted'].sum() * M).reset_index()[['pred_code','mlt']]
# A = (clf['Y_pred'].query('imp==0')['predicted'].sum() * M).rindex('pred_code', bare=True).sindex('pred_code')#.values
A = (clf['Y_pred'].query('imp==0')['predicted'].sum() * M).rsindex('pred_code').squeeze().to_dict()
# {a['pred_code']: a['mlt'] for a in A}
# A.to_dict('index')
# # ('index')

In [ ]:
self.get('mlt').reset_index().dtypes

In [ ]:
clf['Y_pred'].prep().join(self.get('mlt').prep()).disp(4)

In [ ]:
import seaborn as sns
sns.kdeplot(clf['rslt']['projection21'])
clf['rslt']['projection21'].describe()

In [ ]:
self.mlt.disp(100)

In [ ]:
clf['rslt']

In [ ]:
1+1

In [ ]:
imp.keys()
# imp

In [ ]:
path = {'nm':'test', 'sub':'test2'}
L = listify(path)
nm = L.pop(0)
(self.root_path / join(L,'/')).with_suffix('.pkl')
path

In [ ]:
4200/10/20

In [ ]:
X_imp.disp(1)

In [ ]:
12800/9
17.95*20*4
21*20*52

In [ ]:
str(trf).replace('\n','').replace(' ','')

In [ ]:
import hashlib
self.trf_list = cartesian({k: uniquify(v, key=str) for k,v in self.trf_grid.items()})
self.trf_list = [ColumnTransformer([(c,t,["__"+c]) for c,t in trf.items() if t not in ['drop',None,'']], remainder='drop', verbose_feature_names_out=False) for trf in self.trf_list]
self.trf_dict = {hashlib.md5(str(trf).encode()).hexdigest(): trf for trf in self.trf_list}
self.trf_dict


# self.trf_list = [uniquify({k:v for k,v in t.items() if v not in ['drop',None,'']}) for t in self.trf_list]
# self.trf_list = [ColumnTransformer([(c,t,["__"+c]) for c,t in trf.items()], remainder='drop', verbose_feature_names_out=False) for trf in self.trf_list]
# self.trf_dict = {hashlib.md5(trf).encode()).hexdigest(): trf for trf in self.trf_list}
# self.trf_dict
# trf_par = self.trf_list[0]
# trf = ColumnTransformer([(c,t,["__"+c]) for c,t in trf_par.items()], remainder='drop', verbose_feature_names_out=False)
# hashlib.md5(str(trf).encode()).hexdigest()
# str(trf).replace('\n','').replace(' ','')
# self.trf_list = [uniquify({k:v for k,v in t.items() if v not in ['drop',None,'']}) for t in self.trf_list]

# w = self.trf_list[0]
# json.dumps(w)
# w = [str({str(k):str(v) for k,v in t.items()}) for t in self.trf_list][0]
# print(w)
# print(hash(w))
# self.trf_list = {hash(stringify(t)): t for t in self.trf_list}
# self.trf_list
# {hash(str(t)):1 for t in self.trf_list}
# self.trf_list[0]
# self.trf_list = [uniquify({k:v for k,v in t.items() if v not in ['drop',None,'']}) for t in self.trf_list]


In [ ]:
# hash(stringify(self.trf_list[0]))
# trf = StandardScaler()
# hash(trf)
hash(str(self.trf_list[0]))

In [ ]:
self.trf_list

In [ ]:
self.get('raw_df')
self.get('reg_df')['cur']
# self.get('X')
# self.get('Y')
# .set_index(['pidm','pred_code','crse_code'])

In [ ]:
from flags import *

@dataclasses.dataclass
class TERM(MyBaseClass):
    cycle_day: int = 0
    term_code: int = 202308
    root_path: str = root_path
    show: set = dataclasses.field(default_factory=set)


    def __post_init__(self):
        self.dependence: dict = {'adm':'raw', 'flg':'raw', 'dst':'raw', 'raw':'term', 'reg':'term'}
        super().__post_init__()
    #     pass
        # print(self.__dict__())
TERM(overwrite={'adm'}).overwrite

In [ ]:
dependence = {'adm':'raw', 'flg':'raw', 'dst':'raw', 'raw':'term', 'reg':'term'}
l = 0
overwrite = {'adm'}
while l < len(overwrite):
    l = len(overwrite)
    overwrite |= {y for x in overwrite for y in setify(dependence[x] if x in dependence else {})}
    # print(self.overwrite)
overwrite

In [ ]:
q = TERM(term_code=202408, overwrite={'adm'})
q.overwrite

In [ ]:
a = {'a'}
a |= {'b'}
a

In [ ]:
dep = {
    'a': {'b','c'},
    'b': {'d','e'},
}
over = {'a'}
l = 0
while l < len(over):
    l = len(over)
    over |= {y for x in over for y in setify(dep[x] if x in dep else {})}
    # if s == len(over):
    #     break
    # s = len(over)
over

In [ ]:
def g(L, D):
    I = intersection(L, D.keys())
    g()
    {g(D[y]) for y in intersection(L, D.keys())}



In [ ]:
a = pathlib.Path('/home/scook/institutional_data_analytics/admitted_matriculation_projection/admitted_matriculation_predictor/src/dst')
a = pathlib.Path(a)

In [ ]:
from setup import *
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/admitted_matriculation_predictor/src/dst'
dst = read(fn+'.parq')
write(fn+'.pkl', dst)
# fn.w
# .with_suffix('.pkl')